# Capstone 2
## Supervised Learning


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ds_useful as ds
import requests

[My Useful Data Science Functions](https://github.com/cobyoram/python-for-data-scientists/blob/master/ds_useful.py)

In [20]:
def sort_nearest_locations(locations, latitude, longitude):
    locations['DISTANCE'] = np.sqrt((locations['LATITUDE'] - latitude)**2 + (locations['LONGITUDE'] - longitude)**2)
    locations = locations.sort_values(by='DISTANCE', ascending=True)
    return locations

def get_nearest_df(base_url, years, locations, print_log=False):
    for i in range(len(locations)):
        station_id = locations.iloc[i,0]
        try:
            data = pd.DataFrame()
            for year in years:
                current_url = '{}{}/{}'.format(base_url, year, station_id)
                if print_log:
                    print(current_url)
                df = pd.read_csv(current_url, na_values=[9999.9, 999.9, 99.99])
                data = pd.concat([data, df], axis=0)
            return data
        except:
            if print_log:
                print('Error: unable to find', station_id, 'data for year', year, 'searching next_nearest')
            continue
        # except:
        #     print('Error retrieving one or more of the datasets at station_id', station_id)
        #     continue

In [21]:
co2_base_url = 'https://www.ncei.noaa.gov/data/carbon-cycle-and-greenhouse-gases-esrl-gmd/archive/obs/co2/'
co2_datasets = {
    'South Pole': 'ESRL-GMD-CO2_spo_surface-insitu_1_ccgg_DailyData_s19750101_e20161231_c20171002.txt',
    'American Samoa':'ESRL-GMD-CO2_smo_surface-insitu_1_ccgg_DailyData_s19760101_e20161231_c20171002.txt',
    'Hawaii': 'ESRL-GMD-CO2_mlo_surface-insitu_1_ccgg_DailyData_s19730101_e20161231_c20171002.txt',
    'Alaska': 'ESRL-GMD-CO2_brw_surface-insitu_1_ccgg_DailyData_s19730101_e20161231_c20171002.txt'}

def get_co2_data(base_url, fascility):
    url = co2_base_url + fascility
    columns=['site_code', 'year', 'month', 'day', 'hour', 'minute', 'second', 'value', 'value_unc',
     'nvalue', 'latitude', 'longitude', 'altitude', 'elevation', 'intake_height', 'instrument', 'qcflag']
    return pd.read_table(url, sep=' ', comment='#', names=columns, na_values=[-999.99, -99.99, -9])

co2_data = get_co2_data(co2_base_url, co2_datasets['South Pole'])

In [22]:
co2_data

,site_code,year,month,day,hour,minute,second,value,value_unc,nvalue,latitude,longitude,altitude,elevation,intake_height,instrument,qcflag
0,SPO,1975,1,1,0,0,0,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..
1,SPO,1975,1,2,0,0,0,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..
2,SPO,1975,1,3,0,0,0,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..
3,SPO,1975,1,4,0,0,0,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..
4,SPO,1975,1,5,0,0,0,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15336,SPO,2016,12,27,0,0,0,401.37,0.03,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...
15337,SPO,2016,12,28,0,0,0,401.42,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...
15338,SPO,2016,12,29,0,0,0,401.41,0.04,22,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...
15339,SPO,2016,12,30,0,0,0,401.43,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...


In [23]:
locations = pd.read_csv('station_locations.csv').drop('Unnamed: 0', axis=1)

In [24]:
sorted_locations = sort_nearest_locations(locations, co2_data['latitude'].iloc[0], co2_data['longitude'].iloc[0])
sorted_locations.head()

,STATION,LATITUDE,LONGITUDE,DISTANCE
10931,89013099999.csv,-82.766667,-13.050000,13.787483
10933,89022099999.csv,-75.610000,-26.272222,14.445219
10934,89034099999.csv,-77.866667,-34.616667,15.591658
10932,89014099999.csv,-73.050000,-13.383333,20.419725
10930,89009090001.csv,-90.000000,0.000000,24.800008


In [25]:
temp_base_url = 'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/'
years = np.arange(1980, 2017, 1)

temp_data = get_nearest_df(temp_base_url, years, sorted_locations, print_log=True)

https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1980/89013099999.csv
Error: unable to find 89013099999.csv data for year 1980 searching next_nearest
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1980/89022099999.csv
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1981/89022099999.csv
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1982/89022099999.csv
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1983/89022099999.csv
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1984/89022099999.csv
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1985/89022099999.csv
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1986/89022099999.csv
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1987/89022099999.csv
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1988/89022099999.csv
https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/1989/8

In [26]:
temp_data['datetime'] = pd.to_datetime(temp_data['DATE'])
temp_data.drop('DATE', axis=1, inplace=True)
temp_data.head()

,STATION,LATITUDE,LONGITUDE,ELEVATION,NAME,TEMP,TEMP_ATTRIBUTES,DEWP,DEWP_ATTRIBUTES,SLP,...,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT,datetime
0,89022099999,-75.61,-26.272222,30.0,"HALLEY, AY",26.6,5,24.1,5,990.5,...,NaN,30.2,*,15.8,,0.0,I,NaN,100000,1980-01-01
1,89022099999,-75.61,-26.272222,30.0,"HALLEY, AY",24.2,6,19.7,6,986.2,...,NaN,28.4,,23.0,,NaN,,NaN,1000,1980-01-04
2,89022099999,-75.61,-26.272222,30.0,"HALLEY, AY",26.6,4,NaN,0,991.4,...,NaN,30.2,*,21.2,,NaN,,NaN,1000,1980-01-05
3,89022099999,-75.61,-26.272222,30.0,"HALLEY, AY",21.9,5,19.0,5,988.7,...,NaN,26.6,*,8.6,,0.0,I,NaN,0,1980-01-07
4,89022099999,-75.61,-26.272222,30.0,"HALLEY, AY",17.6,7,14.8,7,993.7,...,NaN,30.2,*,10.4,,0.0,I,NaN,0,1980-01-08


In [27]:
co2_data['datetime'] = pd.to_datetime(co2_data[['year', 'month', 'day', 'hour', 'minute', 'second']])
co2_data.drop(['year', 'month', 'day', 'hour', 'minute', 'second'], axis=1, inplace=True)
co2_data

,site_code,value,value_unc,nvalue,latitude,longitude,altitude,elevation,intake_height,instrument,qcflag,datetime
0,SPO,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..,1975-01-01
1,SPO,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..,1975-01-02
2,SPO,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..,1975-01-03
3,SPO,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..,1975-01-04
4,SPO,NaN,NaN,0,-89.98,-24.8,2820.00,2810.0,10.00,NaN,*..,1975-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...
15336,SPO,401.37,0.03,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...,2016-12-27
15337,SPO,401.42,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...,2016-12-28
15338,SPO,401.41,0.04,22,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...,2016-12-29
15339,SPO,401.43,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...,2016-12-30


In [28]:
combined_data = co2_data.merge(temp_data, on='datetime')
combined_data

,site_code,value,value_unc,nvalue,latitude,longitude,altitude,elevation,intake_height,instrument,...,MXSPD,GUST,MAX,MAX_ATTRIBUTES,MIN,MIN_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNDP,FRSHTT
0,SPO,336.30,0.07,23,-89.98,-24.8,2823.50,2810.0,13.50,NaN,...,12.0,NaN,30.2,*,15.8,,0.0,I,NaN,100000
1,SPO,336.19,0.08,22,-89.98,-24.8,2823.50,2810.0,13.50,NaN,...,20.0,NaN,28.4,,23.0,,NaN,,NaN,1000
2,SPO,336.22,0.08,15,-89.98,-24.8,2823.50,2810.0,13.50,NaN,...,24.1,NaN,30.2,*,21.2,,NaN,,NaN,1000
3,SPO,336.01,0.04,24,-89.98,-24.8,2823.50,2810.0,13.50,NaN,...,15.9,NaN,26.6,*,8.6,,0.0,I,NaN,0
4,SPO,336.16,0.03,20,-89.98,-24.8,2823.50,2810.0,13.50,NaN,...,15.0,NaN,30.2,*,10.4,,0.0,I,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11847,SPO,401.37,0.03,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...,8.9,NaN,28.0,*,10.8,,NaN,,NaN,1000
11848,SPO,401.42,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...,15.9,NaN,28.0,*,13.3,,0.0,I,NaN,0
11849,SPO,401.41,0.04,22,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...,8.9,NaN,28.0,,8.8,,0.0,I,NaN,100000
11850,SPO,401.43,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,...,15.9,NaN,26.6,*,11.1,,0.0,I,NaN,100000


In [29]:
from scipy import interpolate

def get_categorical_columns(df, unique_thresh=10, non_binary=False):
    columns = []
    for col in df.columns:
        if df[col].nunique() < unique_thresh:
            if non_binary and df[col].nunique() < 2:
                continue
            columns.append(col)
    return columns

cat_cols = get_categorical_columns(combined_data.select_dtypes('object'), non_binary=True)
dums = pd.get_dummies(combined_data, columns=cat_cols, drop_first=True)
dums = pd.get_dummies(dums, columns=dums.drop(['datetime'], axis=1).select_dtypes('object').columns, drop_first=True)
dums

,value,value_unc,nvalue,latitude,longitude,altitude,elevation,intake_height,instrument,datetime,...,PRCP,SNDP,FRSHTT,qcflag_...,MAX_ATTRIBUTES_*,MIN_ATTRIBUTES_*,PRCP_ATTRIBUTES_A,PRCP_ATTRIBUTES_G,PRCP_ATTRIBUTES_H,PRCP_ATTRIBUTES_I
0,336.30,0.07,23,-89.98,-24.8,2823.50,2810.0,13.50,NaN,1980-01-01,...,0.0,NaN,100000,1,1,0,0,0,0,1
1,336.19,0.08,22,-89.98,-24.8,2823.50,2810.0,13.50,NaN,1980-01-04,...,NaN,NaN,1000,1,0,0,0,0,0,0
2,336.22,0.08,15,-89.98,-24.8,2823.50,2810.0,13.50,NaN,1980-01-05,...,NaN,NaN,1000,1,1,0,0,0,0,0
3,336.01,0.04,24,-89.98,-24.8,2823.50,2810.0,13.50,NaN,1980-01-07,...,0.0,NaN,0,1,1,0,0,0,0,1
4,336.16,0.03,20,-89.98,-24.8,2823.50,2810.0,13.50,NaN,1980-01-08,...,0.0,NaN,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11847,401.37,0.03,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,2016-12-27,...,NaN,NaN,1000,1,1,0,0,0,0,0
11848,401.42,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,2016-12-28,...,0.0,NaN,0,1,1,0,0,0,0,1
11849,401.41,0.04,22,-89.98,-24.8,2821.28,2810.0,11.28,NaN,2016-12-29,...,0.0,NaN,100000,1,0,0,0,0,0,1
11850,401.43,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,NaN,2016-12-30,...,0.0,NaN,100000,1,1,0,0,0,0,1


In [30]:
missings = ds.missingness_summary(dums)
miss_cols = missings.loc[missings > 90].index
print(miss_cols)
dums.drop(miss_cols, axis=1, inplace=True)
dums

Index(['SNDP', 'instrument', 'GUST'], dtype='object')


,value,value_unc,nvalue,latitude,longitude,altitude,elevation,intake_height,datetime,STATION,...,MIN,PRCP,FRSHTT,qcflag_...,MAX_ATTRIBUTES_*,MIN_ATTRIBUTES_*,PRCP_ATTRIBUTES_A,PRCP_ATTRIBUTES_G,PRCP_ATTRIBUTES_H,PRCP_ATTRIBUTES_I
0,336.30,0.07,23,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-01,89022099999,...,15.8,0.0,100000,1,1,0,0,0,0,1
1,336.19,0.08,22,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-04,89022099999,...,23.0,NaN,1000,1,0,0,0,0,0,0
2,336.22,0.08,15,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-05,89022099999,...,21.2,NaN,1000,1,1,0,0,0,0,0
3,336.01,0.04,24,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-07,89022099999,...,8.6,0.0,0,1,1,0,0,0,0,1
4,336.16,0.03,20,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-08,89022099999,...,10.4,0.0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11847,401.37,0.03,23,-89.98,-24.8,2821.28,2810.0,11.28,2016-12-27,89022099999,...,10.8,NaN,1000,1,1,0,0,0,0,0
11848,401.42,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,2016-12-28,89022099999,...,13.3,0.0,0,1,1,0,0,0,0,1
11849,401.41,0.04,22,-89.98,-24.8,2821.28,2810.0,11.28,2016-12-29,89022099999,...,8.8,0.0,100000,1,0,0,0,0,0,1
11850,401.43,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,2016-12-30,89022099999,...,11.1,0.0,100000,1,1,0,0,0,0,1


In [31]:
for col in missings.loc[missings > 0].drop(miss_cols).index:
    dums[col] = dums[col].interpolate(method='linear', limit_direction='both')
dums

,value,value_unc,nvalue,latitude,longitude,altitude,elevation,intake_height,datetime,STATION,...,MIN,PRCP,FRSHTT,qcflag_...,MAX_ATTRIBUTES_*,MIN_ATTRIBUTES_*,PRCP_ATTRIBUTES_A,PRCP_ATTRIBUTES_G,PRCP_ATTRIBUTES_H,PRCP_ATTRIBUTES_I
0,336.30,0.07,23,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-01,89022099999,...,15.8,0.0,100000,1,1,0,0,0,0,1
1,336.19,0.08,22,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-04,89022099999,...,23.0,0.0,1000,1,0,0,0,0,0,0
2,336.22,0.08,15,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-05,89022099999,...,21.2,0.0,1000,1,1,0,0,0,0,0
3,336.01,0.04,24,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-07,89022099999,...,8.6,0.0,0,1,1,0,0,0,0,1
4,336.16,0.03,20,-89.98,-24.8,2823.50,2810.0,13.50,1980-01-08,89022099999,...,10.4,0.0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11847,401.37,0.03,23,-89.98,-24.8,2821.28,2810.0,11.28,2016-12-27,89022099999,...,10.8,0.0,1000,1,1,0,0,0,0,0
11848,401.42,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,2016-12-28,89022099999,...,13.3,0.0,0,1,1,0,0,0,0,1
11849,401.41,0.04,22,-89.98,-24.8,2821.28,2810.0,11.28,2016-12-29,89022099999,...,8.8,0.0,100000,1,0,0,0,0,0,1
11850,401.43,0.04,23,-89.98,-24.8,2821.28,2810.0,11.28,2016-12-30,89022099999,...,11.1,0.0,100000,1,1,0,0,0,0,1


In [32]:
repeats = ds.repeats_summary(dums, value_agg='mode')
rep_cols = repeats.loc[repeats == 100]

date_cols = ['datetime']
repeat_cols = list(rep_cols.index)
drop_cols = repeat_cols + date_cols

dum_feats = dums.drop(drop_cols, axis=1)
dum_feats.rename(lambda x: x.lower(), axis=1, inplace=True)
dum_feats

,value,value_unc,nvalue,altitude,intake_height,temp,temp_attributes,dewp,dewp_attributes,slp,...,min,prcp,frshtt,qcflag_...,max_attributes_*,min_attributes_*,prcp_attributes_a,prcp_attributes_g,prcp_attributes_h,prcp_attributes_i
0,336.30,0.07,23,2823.50,13.50,26.6,5,24.10,5,990.5,...,15.8,0.0,100000,1,1,0,0,0,0,1
1,336.19,0.08,22,2823.50,13.50,24.2,6,19.70,6,986.2,...,23.0,0.0,1000,1,0,0,0,0,0,0
2,336.22,0.08,15,2823.50,13.50,26.6,4,19.35,0,991.4,...,21.2,0.0,1000,1,1,0,0,0,0,0
3,336.01,0.04,24,2823.50,13.50,21.9,5,19.00,5,988.7,...,8.6,0.0,0,1,1,0,0,0,0,1
4,336.16,0.03,20,2823.50,13.50,17.6,7,14.80,7,993.7,...,10.4,0.0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11847,401.37,0.03,23,2821.28,11.28,23.5,24,21.00,24,993.1,...,10.8,0.0,1000,1,1,0,0,0,0,0
11848,401.42,0.04,23,2821.28,11.28,22.7,24,19.30,24,988.6,...,13.3,0.0,0,1,1,0,0,0,0,1
11849,401.41,0.04,22,2821.28,11.28,18.6,24,16.70,24,989.4,...,8.8,0.0,100000,1,0,0,0,0,0,1
11850,401.43,0.04,23,2821.28,11.28,20.1,24,17.60,24,987.7,...,11.1,0.0,100000,1,1,0,0,0,0,1


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

target = 'value'

X = dum_feats.drop(target, axis=1)
Y = dum_feats[target]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.1)

rfr = RandomForestRegressor(max_depth=8)
rfr.fit(X_train, Y_train)

print(rfr.score(X_train, Y_train))
print(rfr.score(X_test, Y_test))


0.9568005079737143
0.9488732963672345


In [34]:
def feature_selection(data):
    cat_cols = get_categorical_columns(combined_data.select_dtypes('object'), non_binary=True)
    dums = pd.get_dummies(combined_data, columns=cat_cols, drop_first=True)
    dums = pd.get_dummies(dums, columns=dums.drop(['datetime'], axis=1).select_dtypes('object').columns, drop_first=True)

    missings = ds.missingness_summary(dums)
    miss_cols = missings.loc[missings > 90].index
    dums.drop(miss_cols, axis=1, inplace=True)

    for col in missings.loc[missings > 0].drop(miss_cols).index:
        dums[col] = dums[col].interpolate(method='linear', limit_direction='both')

    repeats = ds.repeats_summary(dums, value_agg='mode')
    rep_cols = repeats.loc[repeats == 100]

    date_cols = ['datetime']
    repeat_cols = list(rep_cols.index)
    drop_cols = repeat_cols + date_cols

    dum_feats = dums.drop(drop_cols, axis=1)
    dum_feats.rename(lambda x: x.lower(), axis=1, inplace=True)

    return dum_feats

def get_clean_combined_near_data(co2_base_url, temp_base_url, locations, fascility, years):
    co2_data = get_co2_data(co2_base_url, fascility)
    sorted_locations = sort_nearest_locations(locations, co2_data['latitude'].iloc[0], co2_data['longitude'].iloc[0])
    temp_data = get_nearest_df(temp_base_url, years, sorted_locations)

    temp_data['datetime'] = pd.to_datetime(temp_data['DATE'])
    temp_data.drop('DATE', axis=1, inplace=True)

    co2_data['datetime'] = pd.to_datetime(co2_data[['year', 'month', 'day', 'hour', 'minute', 'second']])
    co2_data.drop(['year', 'month', 'day', 'hour', 'minute', 'second'], axis=1, inplace=True)

    combined_data = co2_data.merge(temp_data, on='datetime')

    features = feature_selection(combined_data)

    return features

def get_XY_set(co2_base_url, temp_base_url, locations, fascility, years):
    features = get_clean_combined_near_data(co2_base_url, temp_base_url, locations, fascility, years)
    target = 'value'

    X = features.drop(target, axis=1)
    Y = features[target]
    
    return X, Y

In [35]:
datasets = {}
for key in co2_datasets.keys():
    fascility = co2_datasets[key]
    X, Y = get_XY_set(co2_base_url, temp_base_url, locations, fascility, years)
    datasets[key] = {'X': X, 'Y': Y}
datasets

{'South Pole': {'X':        value_unc  nvalue  altitude  intake_height  temp  temp_attributes  \
  0           0.07      23   2823.50          13.50  26.6                5   
  1           0.08      22   2823.50          13.50  24.2                6   
  2           0.08      15   2823.50          13.50  26.6                4   
  3           0.04      24   2823.50          13.50  21.9                5   
  4           0.03      20   2823.50          13.50  17.6                7   
  ...          ...     ...       ...            ...   ...              ...   
  11847       0.03      23   2821.28          11.28  23.5               24   
  11848       0.04      23   2821.28          11.28  22.7               24   
  11849       0.04      22   2821.28          11.28  18.6               24   
  11850       0.04      23   2821.28          11.28  20.1               24   
  11851       0.07      23   2821.28          11.28  21.1               24   
  
          dewp  dewp_attributes    slp  sl

In [36]:
# fascility = co2_datasets['Hawaii']
# X, Y = get_XY_set(co2_base_url, temp_base_url, locations, fascility, years)
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.1)
# print(rfr.score(X_test, Y_test))